# Introduction to Inversions with HAZEL2 

Shuo Wang

Dept. of Astronomy, NMSU

DKIST Ambassador

[5th NCSP DKIST Data-Training Workshop: He I Diagnostics in the Solar Atmosphere](https://nso.edu/ncsp/ncsp-workshop/hei_diagnostics/)

In the presentation yesterday, we saw a graph with two directed edges representing synthesis and inversion. Together they form a **loop**. Let's implement the loop by doing the following steps:

1. Set a one-slab model. 
2. Do **synthesis** to generate the Stokes profiles.
3. Add noise to the Stokes profiles to simulate observations.
4. Do **inversion**.
5. Compare inversion results with the model.

The following code pieces together the code snippets explained in the previous presentations. Explanation of the code can be found in the [slides](http://astronomy.nmsu.edu/shuowang/dkistWorkshop5/).

## Synthesis

In [ ]:
import hazel
import numpy as np
import h5py
import matplotlib.pyplot as plt

Set up model. LOS: $\theta, \phi, \gamma$.

In [ ]:
mod = hazel.Model(working_mode='synthesis')
mod.add_spectral({'Name': 'spec1', 'Wavelength': [10828, 10831, 50], 'topology': 'ch1',
    'LOS': [0,0,90], 'Boundary condition': [1,0,0,0]})
mod.add_chromosphere({'Name': 'ch1', 'Spectral region': 'spec1', 'Height': 3, 
                          'Line': '10830', 'Wavelength': [10828, 10831]})
mod.setup()

Input: Bx, By, Bz, $\tau$, v, $\delta$v, $\beta$, a, ff.

In [ ]:
mod.atmospheres['ch1'].set_parameters([-200, 50, 100, 1, -3, 6, 1, 0.5], 1)
mod.synthesize()

Plot spectral lines.

In [ ]:
iq = 'IQUV'
ms = mod.spectrum['spec1']
plt.figure(figsize = (12,8))
for i in range(4):
    plt.subplot(221+i)
    plt.plot(ms.wavelength_axis, ms.stokes[i])
    plt.xlabel('Wavelength [$\AA$]')
    plt.ylabel(iq[i]+'/Ic')

## Inversion

### Prepare input files:

**1. data file**

In [ ]:
noise = 2e-4
stokes = np.random.normal(loc=ms.stokes, scale=noise, size=ms.stokes.shape) # Add noise
f = open('10830aStokes.1d', 'wb')
f.write(b'# LOS theta_LOS, phi_LOS, gamma_LOS\n')
f.write(b'0 0 90\n')
f.write(b'\n')
f.write(b'# Boundary condition I/Ic(mu=1), Q/Ic(mu=1), U/Ic(mu=1), V/Ic(mu=1)\n')
f.write(b'1 0 0 0\n')
f.write(b'\n')
f.write(b'# SI SQ SU SV sigmaI sigmaQ sigmaU sigmaV\n')
tmp = np.vstack([stokes, noise*np.ones((4,len(ms.wavelength_axis)))])
np.savetxt(f, tmp.T)
f.close()
%cat  -n 10830aStokes.1d

**2. model file**

In [ ]:
%cat -n model_chromosphere.1d

**3. configuration file**

In [ ]:
%cat -n configSimple.ini

### Invert

In [ ]:
modi = hazel.Model('configSimple.ini', working_mode='inversion',verbose=1)
modi.read_observation()
modi.open_output()
modi.invert()
modi.write_output()
modi.close_output()

### Read results.

In [ ]:
res = h5py.File('output.h5', 'r')  # explicitly close when no longer in use.
sto = ['Bx','By','Bz','tau','v','deltav','beta','a','ff']
stp = ''
for i in sto:
    sti = res['ch1'][i][0,0,0]
    stp += ', '+i+':'+f'{sti:.2f}'
chi2 = res['spec1']['chi2'][0,0,0]
print(stp[2:]+', chi2:'+f'{chi2:.2f}')

Compare with input: 

[Bx, By, Bz, $\tau$, v, $\delta$v, $\beta$, a], ff

[-200, 50, 100, 1, -3, 6, 1, 0.5], 1)

In [ ]:
plt.figure(figsize=(12,8))
for i in range(4):
    plt.subplot(221+i)
    plt.plot(ms.wavelength_axis, ms.stokes[i],label='syn')
    plt.plot(ms.wavelength_axis, stokes[i],'.',label='syn + noise')
    plt.plot(ms.wavelength_axis, res['spec1']['stokes'][0,0,i],label='inv')
    plt.legend()
    plt.xlabel('Wavelength [$\AA$]')
    plt.ylabel(iq[i]+'/Ic')
res.close() # close when no longer in use.